In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [18]:
directories = ['tw/pb_ratio', "tw/price"]
parquet_files = [os.path.join(directories[1], f) for f in os.listdir(directories[1]) if f.endswith(".parquet")] 
dfs = []
parquet_files.remove("tw/price/2021.parquet")
parquet_files.remove("tw/price/2019.parquet")
# parquet_files.remove("tw/price/2018.parquet")
print(parquet_files)
for file in parquet_files:
    df = pd.read_parquet(file)
    dfs.append(df)
df_concat = pd.concat(dfs)
pivoted = df_concat.pivot(index='da', values="cl", columns="ticker")



['tw/price/2023.parquet', 'tw/price/2022.parquet', 'tw/price/2020.parquet', 'tw/price/2024.parquet', 'tw/price/2018.parquet']


In [57]:
b = a.iloc[5600:5900]
i = 1
b[50*i:50*(i+1)]

,da,vol(volume),vol(turnover),op,cl,lo,hi,cl-op,vol(amount),ticker
17,2021-11-24,7645834,1855554920,235.00,248.00,235.00,245.00,+16.50,5799,2383
18,2021-11-25,13354418,3499820410,252.00,269.50,251.50,269.50,+24.50,11216,2383
19,2021-11-26,10346820,2751245958,261.50,272.50,261.00,272.50,+3.00,8292,2383
20,2021-11-29,7369631,2028123832,267.00,280.50,266.50,280.00,+7.50,8431,2383
21,2021-11-30,6583983,1862776317,290.50,293.50,276.00,280.00,0.00,5939,2383
0,2021-04-01,915542,154051771,170.00,170.00,167.50,168.50,-0.50,830,2383
1,2021-04-06,1580145,264899606,169.50,169.50,167.00,167.50,-1.00,1378,2383
2,2021-04-07,1299211,219125270,169.00,169.50,167.50,169.00,+1.50,992,2383
3,2021-04-08,2425647,414121736,170.00,172.00,168.50,171.50,+2.50,1891,2383
4,2021-04-09,2897736,493460046,173.00,173.00,168.50,169.00,-2.50,2385,2383


In [58]:
b = a.iloc[5600:5900]
for i in list(set(b['da'])):
    print(b[b.da==i])

          da vol(volume) vol(turnover)      op      cl      lo      hi  cl-op  \
1 2021-07-02     5759529    1232990402  213.50  218.00  210.50  216.50  +6.50   

  vol(amount) ticker  
1        3828   2383  
           da vol(volume) vol(turnover)      op      cl      lo      hi  \
19 2021-11-26    10346820    2751245958  261.50  272.50  261.00  272.50   
19 2021-11-26    10346820    2751245958  261.50  272.50  261.00  272.50   

    cl-op vol(amount) ticker  
19  +3.00        8292   2383  
19  +3.00        8292   2383  
           da vol(volume) vol(turnover)      op      cl      lo      hi  \
10 2021-10-18     1136837     237688349  211.00  213.00  206.50  207.00   

    cl-op vol(amount) ticker  
10  -4.50         995   2383  
          da vol(volume) vol(turnover)      op      cl      lo      hi  cl-op  \
4 2021-10-07     3886538     820054570  203.50  215.50  203.50  211.50  +8.50   

  vol(amount) ticker  
4        2972   2383  
          da vol(volume) vol(turnover)      op    

In [36]:
a = pd.read_parquet("tw/price/2021.parquet")
for i in range(5800, 0, -100):
    print(i)
    b = a.iloc[i:5900, :]
    pivoted = b.pivot(index='da', values="cl", columns="ticker")


5800
5700
5600


ValueError: Index contains duplicate entries, cannot reshape

In [33]:
values_dict = {
    "cl": "close",
    "op": "open",
    "vol(turnover)": "volume",
    'vol(volume)': "volume(share)",
    "yield": "yield",
    "pe_ratio": "pe_ratio",
    "pb_ratio": "pb_ratio",
}
values = 'vol(turnover)'
pivoted = df_concat.pivot(index='da', values=values, columns="ticker")
pivoted.replace("--", None, inplace=True)
pivoted = pivoted.ffill()
pivoted = pivoted.astype(float)
pivoted.astype(float).to_parquet(f"tw/pdata/{values_dict[values]}.parquet")
pivoted_pct = pivoted.pct_change().dropna(how="all")
pivoted_pct.to_parquet(f'tw/pdata/{values_dict[values]}_pct.parquet')